# Mutation rates analysis

In this notebook, we calculate mutation rates in PAN027 assembly in regard to GIAB stratification of easy (high-confidence) and difficult regions,
as well as problematic regions in PAN027.

In [1]:
import pandas as pd
import os
import tempfile
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
import shutil

In [2]:
# ---------- INPUTS ----------
variants_vcf: str = "variants.vcf.gz"           # concatenated variant files from the second step of the switch_errors pipeline,
                                                #  filtered not to include variants coming from problematic regions
sw_prone_bed: str = "sw_prone_regions.bed"      # switch error-prone regions detected by switch_errors pipeline
flagger_ont: str = "flagger.PAN027.ONT.bed"     # problematic regions on PAN027 assembly
flagger_hifi: str = "flagger.PAN027.hifi.bed"
problematic: str = "problematic.PAN027.bed"
ignore_snvs_regions: list[str] = []             # ignored BED regions, not used
UNRELIABLE_GRANDPARENTS: str = "problematic_grandparents.vcf.gz"    # variants originating from problematic grandparents' regions
mat_easy: str = "chm13_to_PAN027_mat.easy.mapped.PAN027names.bed"   # easy regions on maternal haplotype of PAN027, creating by lifting-over
                                                                    #  CHM13_notinalldifficultregions.bed.gz to PAN027 maternal
                                                                    #  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v3.6/CHM13@all/Union/CHM13_notinalldifficultregions.bed.gz
mat_hard: str = "chm13_to_PAN027_mat.hard.PAN027names.bed"          # difficult regions on maternal haplotype of PAN027, created by complementing
                                                                    #  easy regions
pat_easy: str = "chm13_to_PAN027_pat.easy.mapped.PAN027names.bed"   # easy regions on paternal haplotype of PAN027, created as above
pat_hard: str = "chm13_to_PAN027_pat.hard.PAN027names.bed"          # difficult regions on paternal haplotype of PAN027, created as above
all_blocks: str = "all_blocks.bed"  # regions covered by transmitted blocks, extracted from results.json of swirtch_errors pipeline

chromosomes: list[str] = [f"chr{i}" for i in range(1, 23)] + ["chrX"]
PAD:int = 20_000  # padding (slope) for problematic regions in PAN027
THREADS: int = 8  # parallelism

In [4]:
def run(cmd: list[str], capture: bool = False, check: bool = True, shell: bool = True) -> str | None:
    """
    Safely run shell command

    Parameters
    ----------
    cmd        - command to be executed
    capture    - if True, capture and return stdout and stderr
    check      - if True, verify command has executed successfully (exit code 0), otherwise throw exception
    shell      - if True, execute in local shell

    Returns
    -------
    String stdout + stderr if capture is True, None otherwise
    """

    if capture:
        out = subprocess.check_output(cmd, shell=shell, stderr=subprocess.STDOUT)
        return out.decode().strip()
    else:
        subprocess.run(cmd, shell=shell, check=check)

def ensure_file_exists(path: str) -> None:
    """
    Verifies that file exists, throws otherwise

    Parameters
    ----------
    path - to be verified

    Returns
    -------
    None
    """
    if not Path(path).exists():
        raise FileNotFoundError(f"Required file not found: {path}")

In [5]:
def filter_inputs_for_chrom(chrom: str, tmpdir: Path) -> tuple[Path, Path]:
    """
    Filter variants.vcf.gz and all_blocks.bed for given chromosome, also with removing of unreliable variants
    Returns per-chromosome variants file and all_blocks file
    """

    if chrom is None:
        return variants_vcf, all_blocks  # use originals

    chr_variants: Path = tmpdir / f"variants_{chrom}.vcf.gz"
    chr_blocks: Path = tmpdir / f"blocks_{chrom}.bed"

    # remove unreliable variants
    filtered_vcf: Path = tmpdir / f"variants_filtered_unreliable.vcf.gz"
    cmd_filter_unreliable = (
        f"bcftools isec -C -O z -o {filtered_vcf} "
        f"{variants_vcf} {UNRELIABLE_GRANDPARENTS}" # {UNRELIABLE_CONFIDENCE}
    )
    subprocess.run(cmd_filter_unreliable, shell=True, check=True)
    subprocess.run(f"bcftools index -f {filtered_vcf}", shell=True, check=True)

    # filter by chromosome
    cmd_vcf = (
        f"bcftools view -r PAN027.{chrom}.maternal,PAN027.{chrom}.paternal "
        f"-Oz -o {chr_variants} {filtered_vcf}"
    )
    subprocess.run(cmd_vcf, shell=True, check=True)
    subprocess.run(f"bcftools index -f {chr_variants}", shell=True, check=True)

    # filter BED
    cmd_bed = (
        f"grep -E 'PAN027\\.{chrom}\\.(maternal|paternal)' {all_blocks} > {chr_blocks}"
    )
    subprocess.run(cmd_bed, shell=True, check=True)

    return chr_variants, chr_blocks


In [6]:
def mk_summary_single(tmpdir: Path, variants: Path, allblocks: Path) -> pd.DataFrame:
    """
    Summarize mutation rates per easy/difficult regions for specific input files

    Parameters
    ----------
    tmpdir      - temporary directory for intermediates
    variants    - variant file (possibly per-chromosome)
    allblocks   - transmitted blocks file (possibly per-chromosome)

    Returns
    -------
    dataframe containing mutation rates
    """

    out_prefix: str = "mutation_rates_summary"
    out_tsv: str = f"{tmpdir}/{out_prefix}.tsv"
    
    try:
        # ---------- Build combined exclusion BED ----------
        # 1) pad flagger_ont, flagger_hifi, and problematic
        padded_flagger_ont: Path = tmpdir/"flagger_ont.padded.bed"
        padded_flagger_hifi: Path = tmpdir/"flagger_hifi.padded.bed"
        padded_problematic: Path = tmpdir/"problematic.padded.bed"

        for inp, out in [(flagger_ont, padded_flagger_ont),
                         (flagger_hifi, padded_flagger_hifi),
                         (problematic, padded_problematic)]:
            cmd = (
                f"awk 'BEGIN{{OFS=\"\\t\"}} "
                f"{{s=$2-{PAD}; if(s<0) s=0; print $1, s, $3+{PAD}}}' {inp} | "
                f"sort -k1,1 -k2,2n | bedtools merge -i - > {out}"
            )
            print("[CMD] padding+merge ->", out)
            run(cmd)
        
        # 2) combine sw_prone + padded flaggers + padded problematic
        combined_exclude: Path = tmpdir / "combined_exclude.bed"
        tmp_comb: Path = tmpdir / "combined_exclude.concat.bed"
        with open(tmp_comb, "w") as out_f:
            for src in [sw_prone_bed, padded_flagger_ont, padded_flagger_hifi, padded_problematic]:
                with open(src) as f:
                    for line in f:
                        if line.startswith("#") or not line.strip():
                            continue
                        fields: list[str] = line.strip().split()
                        if len(fields) < 3:
                            continue
                        try:
                            start: int = int(fields[1])
                            end: int = int(fields[2])
                        except ValueError:
                            continue  # skip malformed
                        out_f.write(f"{fields[0]}\t{start}\t{end}\n")
        cmd = (
            f"sort -k1,1 -k2,2n {tmp_comb} > {tmp_comb}.sorted && "
            f"bedtools merge -i {tmp_comb}.sorted > {combined_exclude}"
        )
        print("[CMD] combining excludes ->", combined_exclude)
        run(cmd)
    
        # ---------- Prepare region sets: all / easy / hard, for maternal/paternal/all ----------
        # output: regions_all.bed, regions_mat_easy.bed, regions_mat_hard.bed, regions_pat_easy.bed, regions_pat_hard.bed
        # we are working only on transmitted blocks w.r.t. exclusion created above
        def prepare_region(in_bed: str, out_bed: str) -> None: # intersect with all_blocks, subtract excludes, sort+merge
            cmd = (
                f"bedtools intersect -a {in_bed} -b {allblocks} -u | "
                f"bedtools subtract -a - -b {combined_exclude} | "
                f"sort -k1,1 -k2,2n | bedtools merge -i - > {out_bed}"
            )
            run(cmd)

        def get_bed_total_length(bed_path: str) -> int:
            """
            Calculates total length of regions covered by BED file
            """
            cmd = f"awk '{{s+=$3-$2}} END{{print s+0}}' {bed_path}"
            out = run(cmd, capture=True)
            return int(float(out))
    
        regions = {}
        # maternal easy/hard
        regions['mat_easy'] = tmpdir/"mat_easy.selected.bed"
        regions['mat_hard'] = tmpdir/"mat_hard.selected.bed"
        prepare_region(mat_easy, regions['mat_easy'])
        prepare_region(mat_hard, regions['mat_hard'])
        # paternal easy/hard
        regions['pat_easy'] = tmpdir/"pat_easy.selected.bed"
        regions['pat_hard'] = tmpdir/"pat_hard.selected.bed"
        prepare_region(pat_easy, regions['pat_easy'])
        prepare_region(pat_hard, regions['pat_hard'])
    
        # define maternal-only = mat intersect all_blocks, paternal-only similarly
        mat_union: Path = tmpdir/"mat_union.bed"
        pat_union: Path = tmpdir/"pat_union.bed"
        run(f"cat {regions['mat_easy']} {regions['mat_hard']} | sort -k1,1 -k2,2n | bedtools merge -i - > {mat_union}")
        run(f"cat {regions['pat_easy']} {regions['pat_hard']} | sort -k1,1 -k2,2n | bedtools merge -i - > {pat_union}")
    
        mat_only: Path = tmpdir/"mat_only.bed"
        pat_only: Path = tmpdir/"pat_only.bed"
        run(f"bedtools subtract -a {mat_union} -b {pat_union} | sort -k1,1 -k2,2n | bedtools merge -i - > {mat_only}")
        run(f"bedtools subtract -a {pat_union} -b {mat_union} | sort -k1,1 -k2,2n | bedtools merge -i - > {pat_only}")
    
        all_regions: Path = tmpdir/"all_regions.bed"
        run(f"cat {mat_union} {pat_union} | sort -k1,1 -k2,2n | bedtools merge -i - > {all_regions}")

        mat_len_total: int = get_bed_total_length(mat_only)
        pat_len_total: int = get_bed_total_length(pat_only)

        # build region files to distinct all/easy/hard regions for both maternal and paternal region files
        combos = {}
        for hap_label, hap_union, hap_easy, hap_hard in [
            ("all", str(all_regions), None, None),
            ("maternal", str(mat_union), str(regions['mat_easy']), str(regions['mat_hard'])),
            ("paternal", str(pat_union), str(regions['pat_easy']), str(regions['pat_hard'])),
        ]:
            if hap_label == "all":
                combos[(hap_label, "all")] = str(all_regions)
                merged_easy = tmpdir/"both_easy.bed"
                run(f"cat {regions['mat_easy']} {regions['pat_easy']} | sort -k1,1 -k2,2n | bedtools merge -i - > {merged_easy}")
                merged_hard = tmpdir/"both_hard.bed"
                run(f"cat {regions['mat_hard']} {regions['pat_hard']} | sort -k1,1 -k2,2n | bedtools merge -i - > {merged_hard}")
                combos[(hap_label, "easy")] = str(merged_easy)
                combos[(hap_label, "hard")] = str(merged_hard)
            else:
                combos[(hap_label, "all")] = hap_union
                combos[(hap_label, "easy")] = hap_easy
                combos[(hap_label, "hard")] = hap_hard

        def count_variants_in_region(vcf: str, region_bed: str, variant_type="all"):
            """
            Function to count variants in region with bcftools
            variant_type: "all", "snvs", "indels"
            """

            # remove unreliable variants
            filtered_vcf = tmpdir / f"variants_filtered.vcf.gz"
            cmd_filter_unreliable = (
                f"bcftools isec -w1 -C -O z -o {filtered_vcf} "
                f"{vcf} {UNRELIABLE_GRANDPARENTS}"
            )
            subprocess.run(cmd_filter_unreliable, shell=True, check=True)

            # index
            cmd_index = (
                f"tabix -p vcf -f {filtered_vcf} "
            )
            subprocess.run(cmd_index, shell=True, check=True)
            
            # filter variants of interest
            if variant_type == "all":
                vcf_cmd = f"bcftools view -R {region_bed} -H {filtered_vcf}"
            elif variant_type.lower() in ("snv", "snvs", "snp", "snps"):
                vcf_cmd = f"bcftools view -v snps -R {region_bed} -H {filtered_vcf}"
            elif variant_type.lower() in ("indel", "indels"):
                vcf_cmd = f"bcftools view -v indels -R {region_bed} -H {filtered_vcf}"
            else:
                raise ValueError("unknown variant_type: "+str(variant_type))

            # count variants
            cmd = vcf_cmd + " | wc -l"
            out = run(cmd, capture=True)
            return int(out)
    
        # count mutation rates for all combinations of (maternal, paternal, both) x (easy, difficult, all) x (SNVs, indels, either)
        rows = []
        mutation_type_labels: list[tuple] = [("all","all"), ("snvs","SNVs"), ("indels","Indels")]
        hap_labels: list[str] = ["all", "maternal", "paternal"]
        region_types: list[str] = ["all","easy","hard"]
    
        print("[INFO] Running counts...")
        for hap in hap_labels:
            for mut_key, mut_label in mutation_type_labels:
                for reg in region_types:
                    region_bed = combos[(hap, reg)]
                    try:
                        length_bp: int = get_bed_total_length(region_bed)
                    except subprocess.CalledProcessError:
                        length_bp = 0
                    if length_bp == 0:
                        count = 0
                    else:
                        count = count_variants_in_region(variants, region_bed, mut_key)
                    per_mb: float = float(count) / (length_bp / 1e6) if length_bp>0 else float("nan")
    
                    rows.append({
                        "haplotype_scope": hap,
                        "mutation_type": mut_label,
                        "region_scope": reg,
                        "region_bed": region_bed,
                        "length_bp": length_bp,
                        "variant_count": count,
                        "variants_per_Mb": per_mb,
                        "region_scope_len": mat_len_total if hap == "maternal" else pat_len_total if hap == "paternal" else 0
                    })
                    print(f"[RESULT] hap={hap} mut={mut_label} reg={reg} count={count} len={length_bp} perMb={per_mb:.3f}")
    
        results_df = pd.DataFrame(rows)
        results_df = results_df[[
            "haplotype_scope","region_scope","mutation_type","region_bed","length_bp","variant_count","variants_per_Mb", "region_scope_len"
        ]]

        results_df.to_csv(out_tsv, sep="\t", index=False)
        print(f"[DONE] summary written to {out_tsv}")
        return results_df
    finally:
        print("[INFO] cleaning tmpdir:", tmpdir)
        shutil.rmtree(tmpdir, ignore_errors=True)

In [7]:
def run_single_chromosome(tmp_root: Path, chrom: str) -> pd.DataFrame:
    """
    Run mutation rate summary for one chromosome
    """
    tmpdir = tmp_root / chrom
    tmpdir.mkdir(exist_ok=True)
    chr_variants, chr_blocks = filter_inputs_for_chrom(chrom, tmpdir)

    df = mk_summary_single(tmpdir, chr_variants, chr_blocks)
    df["chromosome"] = chrom
    return df

#### Calculate coarse mutation rates (i.e., without chromosomal differentiation)

In [8]:
base_tmp: str = os.environ.get("SCRATCH", None) or os.environ.get("TMPDIR", None) or "/tmp"
tmpdir: Path = Path(tempfile.mkdtemp(prefix="mutrate_", dir=base_tmp))
print("[INFO] tmpdir:", tmpdir)

mk_summary_single(tmpdir, variants_vcf, all_blocks)

[INFO] tmpdir: /tmp/mutrate_zk4371zv
[CMD] padding+merge -> /tmp/mutrate_zk4371zv/flagger_ont.padded.bed
[CMD] padding+merge -> /tmp/mutrate_zk4371zv/flagger_hifi.padded.bed
[CMD] padding+merge -> /tmp/mutrate_zk4371zv/problematic.padded.bed
[CMD] combining excludes -> /tmp/mutrate_zk4371zv/combined_exclude.bed
[INFO] Running counts (this may take a while depending on VCF size).
[RESULT] hap=all mut=all reg=all count=19539 len=2674270000 perMb=7.306
[RESULT] hap=all mut=all reg=easy count=355 len=2038533185 perMb=0.174
[RESULT] hap=all mut=all reg=hard count=19187 len=635731978 perMb=30.181
[RESULT] hap=all mut=SNVs reg=all count=569 len=2674270000 perMb=0.213
[RESULT] hap=all mut=SNVs reg=easy count=221 len=2038533185 perMb=0.108
[RESULT] hap=all mut=SNVs reg=hard count=348 len=635731978 perMb=0.547
[RESULT] hap=all mut=Indels reg=all count=18970 len=2674270000 perMb=7.094
[RESULT] hap=all mut=Indels reg=easy count=134 len=2038533185 perMb=0.066
[RESULT] hap=all mut=Indels reg=hard co

,haplotype_scope,region_scope,mutation_type,region_bed,length_bp,variant_count,variants_per_Mb,region_scope_len
0,all,all,all,/tmp/mutrate_zk4371zv/all_regions.bed,2674270000,19539,7.306293,0
1,all,easy,all,/tmp/mutrate_zk4371zv/both_easy.bed,2038533185,355,0.174145,0
2,all,hard,all,/tmp/mutrate_zk4371zv/both_hard.bed,635731978,19187,30.180958,0
3,all,all,SNVs,/tmp/mutrate_zk4371zv/all_regions.bed,2674270000,569,0.212768,0
4,all,easy,SNVs,/tmp/mutrate_zk4371zv/both_easy.bed,2038533185,221,0.108411,0
5,all,hard,SNVs,/tmp/mutrate_zk4371zv/both_hard.bed,635731978,348,0.547400,0
6,all,all,Indels,/tmp/mutrate_zk4371zv/all_regions.bed,2674270000,18970,7.093525,0
7,all,easy,Indels,/tmp/mutrate_zk4371zv/both_easy.bed,2038533185,134,0.065734,0
8,all,hard,Indels,/tmp/mutrate_zk4371zv/both_hard.bed,635731978,18839,29.633557,0
9,maternal,all,all,/tmp/mutrate_zk4371zv/mat_union.bed,1527643660,8206,5.371672,1527643660


#### Calculate fine mutation rates (i.e., with chromosomal differentiation)

In [34]:
base_tmp: str = os.environ.get("SCRATCH", None) or os.environ.get("TMPDIR", None) or "/tmp"
tmpdir: Path = Path(tempfile.mkdtemp(prefix="mutrate_", dir=base_tmp))
print("[INFO] tmpdir:", tmpdir)

results: list[dict] = []
with ProcessPoolExecutor(max_workers=16) as ex:
    futures = {ex.submit(run_single_chromosome, tmpdir, chrom): chrom for chrom in chromosomes}
    for fut in as_completed(futures):
        chrom = futures[fut]
        try:
            df = fut.result()
            results.append(df)
            print(f"[OK] {chrom}")
        except Exception as e:
            print(f"[FAIL] {chrom}: {e}")

# concat
final_df = pd.concat(results, ignore_index=True)
final_df.to_csv("mutation_rate_by_chromosome.csv", index=False)
print("[INFO] Final concatenated shape:", final_df.shape)

final_df

[INFO] tmpdir: /tmp/mutrate_896r0tio
[CMD] padding+merge ->[CMD] padding+merge ->  /tmp/mutrate_896r0tio/chr16/flagger_ont.padded.bed/tmp/mutrate_896r0tio/chr12/flagger_ont.padded.bed

[CMD] padding+merge -> /tmp/mutrate_896r0tio/chr13/flagger_ont.padded.bed
[CMD] padding+merge ->[CMD] padding+merge ->[CMD] padding+merge ->[CMD] padding+merge ->[CMD] padding+merge ->[CMD] padding+merge ->  [CMD] padding+merge ->[CMD] padding+merge ->[CMD] padding+merge -> [CMD] padding+merge -> [CMD] padding+merge -> [CMD] padding+merge ->/tmp/mutrate_896r0tio/chr6/flagger_ont.padded.bed[CMD] padding+merge ->/tmp/mutrate_896r0tio/chr5/flagger_ont.padded.bed    /tmp/mutrate_896r0tio/chr10/flagger_ont.padded.bed /tmp/mutrate_896r0tio/chr8/flagger_ont.padded.bed/tmp/mutrate_896r0tio/chr11/flagger_ont.padded.bed 
 
 /tmp/mutrate_896r0tio/chr9/flagger_ont.padded.bed/tmp/mutrate_896r0tio/chr2/flagger_ont.padded.bed/tmp/mutrate_896r0tio/chr4/flagger_ont.padded.bed/tmp/mutrate_896r0tio/chr15/flagger_ont.padded

,haplotype_scope,region_scope,mutation_type,region_bed,length_bp,variant_count,variants_per_Mb,region_scope_len,chromosome
0,all,all,all,/tmp/mutrate_896r0tio/chr16/all_regions.bed,75751317,812,10.719286,0,chr16
1,all,easy,all,/tmp/mutrate_896r0tio/chr16/both_easy.bed,48527966,3,0.061820,0,chr16
2,all,hard,all,/tmp/mutrate_896r0tio/chr16/both_hard.bed,27223351,809,29.717135,0,chr16
3,all,all,SNVs,/tmp/mutrate_896r0tio/chr16/all_regions.bed,75751317,19,0.250821,0,chr16
4,all,easy,SNVs,/tmp/mutrate_896r0tio/chr16/both_easy.bed,48527966,1,0.020607,0,chr16
...,...,...,...,...,...,...,...,...,...
616,paternal,easy,SNVs,/tmp/mutrate_896r0tio/chrX/pat_easy.selected.bed,9722928,0,0.000000,12831596,chrX
617,paternal,hard,SNVs,/tmp/mutrate_896r0tio/chrX/pat_hard.selected.bed,3108668,2,0.643362,12831596,chrX
618,paternal,all,Indels,/tmp/mutrate_896r0tio/chrX/pat_union.bed,12831596,127,9.897444,12831596,chrX
619,paternal,easy,Indels,/tmp/mutrate_896r0tio/chrX/pat_easy.selected.bed,9722928,2,0.205699,12831596,chrX
